## Plotting the location of the lake breeze and analyzing occurrence for a pneumonia front over Chicago.


This example notebook shows how to use ADAM to infer the lake breeze location over Chicago. We choose the pneumonia front over Chicago that occured on April 24, 2025 as our case.


In [61]:
import adam
import pyart
import numpy as np
import matplotlib.pyplot as plt

from scipy.optimize import minimize
from scipy.ndimage import center_of_mass

To begin, we preprocess the radar data to extract the reflectivity fields necessary for lake breeze detection. The ADAM library streamlines this process by converting radar reflectivity fields into ImageNet-compatible images, which are then ready for use with inference models. For batch processing of multiple NEXRAD files, use `adam.io.preprocess_radar_image_batch`; for single images, use `adam.io.preprocess_radar_image`.

By default, `adam.io.preprocess_radar_image_batch` leverages Dask to perform parallel pre-processing on a list of files. If you prefer serial processing, simply set the `parallel` keyword argument to `False`.

In [62]:
rad_scan1 = adam.io.preprocess_radar_image_batch('/Users/rjackson/klot_batch_test/*_*V06', parallel=True)


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119

## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119

## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather ra

If you want to process a single radar image, use 

In [ ]:
rad_scan = adam.io.preprocess_radar_image('KLOT', '2025-04-24T19:42:38')
rad_scan.pyart_object.time

In [ ]:
 np.datetime64(rad_scan.pyart_object.time["units"].split()[2]) == np.datetime64('2025-07-17T16:58:43')

In [63]:
rad_scan = adam.model.infer_lake_breeze_batch(
    rad_scan1, model_name='lakebreeze_model_fcn_resnet50_no_augmentation')

In [64]:
adam.vis.visualize_lake_breeze(rad_scan, vmin=-30, vmax=60, cmap='HomeyerRainbow')

ValueError: Since the input radar_scan represents a batch of radar files, the UTC time in YYYY-MM-DDTHH:MM:SS format must be specified!

## Plot a density plot of the lake breeze occurrence over Chicago for the day